In [1]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [2]:
!unzip -qq "/content/gdrive/MyDrive/demand-forecasting-kernels-only.zip"

In [11]:
!pip install chart-studio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.0 MB/s eta 0:00:00


In [14]:
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras import optimizers
from keras.utils import plot_model
from keras.models import Sequential, Model
from keras.layers import Conv1D, MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import chart_studio.plotly as py
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot

In [15]:
%matplotlib inline
warnings.filterwarnings("ignore")
init_notebook_mode(connected=True)

In [17]:
import tensorflow as tf
tf.random.set_seed(1)

In [18]:
train = pd.read_csv("/content/train.csv",parse_dates=['date'])

In [19]:
train

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [20]:
test = pd.read_csv("/content/test.csv",parse_dates=['date'])

In [25]:
test

,id,date,store,item
0,0,2018-01-01,1,1
1,1,2018-01-02,1,1
2,2,2018-01-03,1,1
3,3,2018-01-04,1,1
4,4,2018-01-05,1,1
...,...,...,...,...
44995,44995,2018-03-27,10,50
44996,44996,2018-03-28,10,50
44997,44997,2018-03-29,10,50
44998,44998,2018-03-30,10,50


In [23]:
print(train['date'].min().date())

2013-01-01


In [24]:
print(train['date'].max().date())

2017-12-31


In [26]:
lag_size = (test['date'].max().date() - train['date'].max().date()).days

In [28]:
print('Max date from train set: %s' % train['date'].max().date())
print('Max date from test set: %s' % test['date'].max().date())
print('Forecast lag size', lag_size)

Max date from train set: 2017-12-31
Max date from test set: 2018-03-31
Forecast lag size 90


In [30]:
daily_sales = train.groupby('date', as_index=False)['sales'].sum()

In [31]:
daily_sales

,date,sales
0,2013-01-01,13696
1,2013-01-02,13678
2,2013-01-03,14488
3,2013-01-04,15677
4,2013-01-05,16237
...,...,...
1821,2017-12-27,20378
1822,2017-12-28,21885
1823,2017-12-29,23535
1824,2017-12-30,24988


In [32]:
store_daily_sales = train.groupby(['store','date'], as_index=False)['sales'].sum()

In [33]:
store_daily_sales

,store,date,sales
0,1,2013-01-01,1316
1,1,2013-01-02,1264
2,1,2013-01-03,1305
3,1,2013-01-04,1452
4,1,2013-01-05,1499
...,...,...,...
18255,10,2017-12-27,2221
18256,10,2017-12-28,2429
18257,10,2017-12-29,2687
18258,10,2017-12-30,2742


In [34]:
item_daily_sales = train.groupby(['item','date'], as_index=False)['sales'].sum()

In [35]:
item_daily_sales

,item,date,sales
0,1,2013-01-01,133
1,1,2013-01-02,99
2,1,2013-01-03,127
3,1,2013-01-04,145
4,1,2013-01-05,149
...,...,...,...
91295,50,2017-12-27,511
91296,50,2017-12-28,587
91297,50,2017-12-29,596
91298,50,2017-12-30,612


In [46]:
train=train[(train['date'] >= '2017-01-01')]

In [47]:
train

,date,store,item,sales
1461,2017-01-01,1,1,19
1462,2017-01-02,1,1,15
1463,2017-01-03,1,1,10
1464,2017-01-04,1,1,16
1465,2017-01-05,1,1,14
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [52]:
train_gp = train.sort_values('date').groupby(['item','store','date'], as_index=False)

In [54]:
train_gp = train_gp.agg({'sales':['mean']})

In [56]:
train_gp.columns = ['item', 'store', 'date', 'sales']
train_gp.head()

,item,store,date,sales
0,1,1,2017-01-01,19.0
1,1,1,2017-01-02,15.0
2,1,1,2017-01-03,10.0
3,1,1,2017-01-04,16.0
4,1,1,2017-01-05,14.0


In [85]:
train_gp

,item,store,date,sales
0,1,1,2017-01-01,19.0
1,1,1,2017-01-02,15.0
2,1,1,2017-01-03,10.0
3,1,1,2017-01-04,16.0
4,1,1,2017-01-05,14.0
...,...,...,...,...
182495,50,10,2017-12-27,63.0
182496,50,10,2017-12-28,59.0
182497,50,10,2017-12-29,74.0
182498,50,10,2017-12-30,62.0


In [89]:
def series_to_supervised(data, window=1,lag=1,dropnan=True):
    cols, names = list(), list()
    for i in range(window,0,-1):
      cols.append(data.shift(i))
      names += [('%s(t-%d)' % (col, i)) for col in data.columns]
    cols.append(data)
    names += [('%s(t)' % (col)) for col in data.columns]
    cols.append(data.shift(-lag))
    names += [('%s(t+%d)' % (col, lag)) for col in data.columns]
    agg = pd.concat(cols, axis=1)
    agg.columns = names
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [90]:
window = 29
lag = lag_size
series = series_to_supervised(train_gp.drop('date', axis=1), window=window, lag=lag)

In [91]:
series

,item(t-29),store(t-29),sales(t-29),item(t-28),store(t-28),sales(t-28),item(t-27),store(t-27),sales(t-27),item(t-26),store(t-26),sales(t-26),item(t-25),store(t-25),sales(t-25),item(t-24),store(t-24),sales(t-24),item(t-23),store(t-23),sales(t-23),item(t-22),store(t-22),sales(t-22),item(t-21),store(t-21),sales(t-21),item(t-20),store(t-20),sales(t-20),item(t-19),store(t-19),sales(t-19),item(t-18),store(t-18),sales(t-18),item(t-17),store(t-17),sales(t-17),item(t-16),store(t-16),sales(t-16),item(t-15),store(t-15),sales(t-15),item(t-14),store(t-14),sales(t-14),item(t-13),store(t-13),sales(t-13),item(t-12),store(t-12),sales(t-12),item(t-11),store(t-11),sales(t-11),item(t-10),store(t-10),sales(t-10),item(t-9),store(t-9),sales(t-9),item(t-8),store(t-8),sales(t-8),item(t-7),store(t-7),sales(t-7),item(t-6),store(t-6),sales(t-6),item(t-5),store(t-5),sales(t-5),item(t-4),store(t-4),sales(t-4),item(t-3),store(t-3),sales(t-3),item(t-2),store(t-2),sales(t-2),item(t-1),store(t-1),sales(t-1),item(t),store(t),sales(t),item(t+90),store(t+90),sales(t+90)
29,1.0,1.0,19.0,1.0,1.0,15.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,14.0,1.0,1.0,24.0,1.0,1.0,14.0,1.0,1.0,20.0,1.0,1.0,18.0,1.0,1.0,11.0,1.0,1.0,14.0,1.0,1.0,17.0,1.0,1.0,7.0,1.0,1.0,16.0,1.0,1.0,29.0,1.0,1.0,15.0,1.0,1.0,14.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,22.0,1.0,1.0,13.0,1.0,1.0,21.0,1.0,1.0,9.0,1.0,1.0,18.0,1.0,1.0,14.0,1.0,1.0,13.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,24.0,1,1,9.0,1.0,1.0,33.0
30,1.0,1.0,15.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,14.0,1.0,1.0,24.0,1.0,1.0,14.0,1.0,1.0,20.0,1.0,1.0,18.0,1.0,1.0,11.0,1.0,1.0,14.0,1.0,1.0,17.0,1.0,1.0,7.0,1.0,1.0,16.0,1.0,1.0,29.0,1.0,1.0,15.0,1.0,1.0,14.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,22.0,1.0,1.0,13.0,1.0,1.0,21.0,1.0,1.0,9.0,1.0,1.0,18.0,1.0,1.0,14.0,1.0,1.0,13.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,24.0,1.0,1.0,9.0,1,1,17.0,1.0,1.0,15.0
31,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,14.0,1.0,1.0,24.0,1.0,1.0,14.0,1.0,1.0,20.0,1.0,1.0,18.0,1.0,1.0,11.0,1.0,1.0,14.0,1.0,1.0,17.0,1.0,1.0,7.0,1.0,1.0,16.0,1.0,1.0,29.0,1.0,1.0,15.0,1.0,1.0,14.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,22.0,1.0,1.0,13.0,1.0,1.0,21.0,1.0,1.0,9.0,1.0,1.0,18.0,1.0,1.0,14.0,1.0,1.0,13.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,24.0,1.0,1.0,9.0,1.0,1.0,17.0,1,1,15.0,1.0,1.0,21.0
32,1.0,1.0,16.0,1.0,1.0,14.0,1.0,1.0,24.0,1.0,1.0,14.0,1.0,1.0,20.0,1.0,1.0,18.0,1.0,1.0,11.0,1.0,1.0,14.0,1.0,1.0,17.0,1.0,1.0,7.0,1.0,1.0,16.0,1.0,1.0,29.0,1.0,1.0,15.0,1.0,1.0,14.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,22.0,1.0,1.0,13.0,1.0,1.0,21.0,1.0,1.0,9.0,1.0,1.0,18.0,1.0,1.0,14.0,1.0,1.0,13.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,24.0,1.0,1.0,9.0,1.0,1.0,17.0,1.0,1.0,15.0,1,1,17.0,1.0,1.0,29.0
33,1.0,1.0,14.0,1.0,1.0,24.0,1.0,1.0,14.0,1.0,1.0,20.0,1.0,1.0,18.0,1.0,1.0,11.0,1.0,1.0,14.0,1.0,1.0,17.0,1.0,1.0,7.0,1.0,1.0,16.0,1.0,1.0,29.0,1.0,1.0,15.0,1.0,1.0,14.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,22.0,1.0,1.0,13.0,1.0,1.0,21.0,1.0,1.0,9.0,1.0,1.0,18.0,1.0,1.0,14.0,1.0,1.0,13.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,24.0,1.0,1.0,9.0,1.0,1.0,17.0,1.0,1.0,15.0,1.0,1.0,17.0,1,1,24.0,1.0,1.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182405,50.0,10.0,79.0,50.0,10.0,89.0,50.0,10.0,96.0,50.0,10.0,92.0,50.0,10.0,102.0,50.0,10.0,60.0,50.0,10.0,81.0,50.0,10.0,79.0,50.0,10.0,78.0,50.0,10.0,97.0,50.0,10.0,93.0,50.0,10.0,96.0,50.0,10.0,74.0,50.0,10.0,70.0,50.0,10.0,84.0,50.0,10.0,90.0,50.0,10.0,94.0,50.0,10.0,97.0,50.0,10.0,107.0,50.0,10.0,61.0,50.0,10.0,84.0,50.0,10.0,72.0,50.0,10.0,89.0,50.0,10.0,97.0,50.0,10.0,97.0,50.0,10.0,99.0,50.0,10.0,63.0,50.0,10.0,79.0,50.0,10.0,80.0,50,10,82.0,50.0,10.0,63.0
182406,50.0,10.0,89.0,50.0,10.0,96.0,50.0,10.0,92.0,50.0,10.0,102.0,50.0,10.0,60.0,50.0,10.0,81.0,50.0,10.0,7

In [95]:
last_item = 'item(t-%d)' % window
last_store = 'store(t-%d)' % window
series = series[(series['store(t)'] == series[last_store])]
series = series[(series['item(t)'] == series[last_item])]

In [96]:
series

,item(t-29),store(t-29),sales(t-29),item(t-28),store(t-28),sales(t-28),item(t-27),store(t-27),sales(t-27),item(t-26),store(t-26),sales(t-26),item(t-25),store(t-25),sales(t-25),item(t-24),store(t-24),sales(t-24),item(t-23),store(t-23),sales(t-23),item(t-22),store(t-22),sales(t-22),item(t-21),store(t-21),sales(t-21),item(t-20),store(t-20),sales(t-20),item(t-19),store(t-19),sales(t-19),item(t-18),store(t-18),sales(t-18),item(t-17),store(t-17),sales(t-17),item(t-16),store(t-16),sales(t-16),item(t-15),store(t-15),sales(t-15),item(t-14),store(t-14),sales(t-14),item(t-13),store(t-13),sales(t-13),item(t-12),store(t-12),sales(t-12),item(t-11),store(t-11),sales(t-11),item(t-10),store(t-10),sales(t-10),item(t-9),store(t-9),sales(t-9),item(t-8),store(t-8),sales(t-8),item(t-7),store(t-7),sales(t-7),item(t-6),store(t-6),sales(t-6),item(t-5),store(t-5),sales(t-5),item(t-4),store(t-4),sales(t-4),item(t-3),store(t-3),sales(t-3),item(t-2),store(t-2),sales(t-2),item(t-1),store(t-1),sales(t-1),item(t),store(t),sales(t),item(t+90),store(t+90),sales(t+90)
29,1.0,1.0,19.0,1.0,1.0,15.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,14.0,1.0,1.0,24.0,1.0,1.0,14.0,1.0,1.0,20.0,1.0,1.0,18.0,1.0,1.0,11.0,1.0,1.0,14.0,1.0,1.0,17.0,1.0,1.0,7.0,1.0,1.0,16.0,1.0,1.0,29.0,1.0,1.0,15.0,1.0,1.0,14.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,22.0,1.0,1.0,13.0,1.0,1.0,21.0,1.0,1.0,9.0,1.0,1.0,18.0,1.0,1.0,14.0,1.0,1.0,13.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,24.0,1,1,9.0,1.0,1.0,33.0
30,1.0,1.0,15.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,14.0,1.0,1.0,24.0,1.0,1.0,14.0,1.0,1.0,20.0,1.0,1.0,18.0,1.0,1.0,11.0,1.0,1.0,14.0,1.0,1.0,17.0,1.0,1.0,7.0,1.0,1.0,16.0,1.0,1.0,29.0,1.0,1.0,15.0,1.0,1.0,14.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,22.0,1.0,1.0,13.0,1.0,1.0,21.0,1.0,1.0,9.0,1.0,1.0,18.0,1.0,1.0,14.0,1.0,1.0,13.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,24.0,1.0,1.0,9.0,1,1,17.0,1.0,1.0,15.0
31,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,14.0,1.0,1.0,24.0,1.0,1.0,14.0,1.0,1.0,20.0,1.0,1.0,18.0,1.0,1.0,11.0,1.0,1.0,14.0,1.0,1.0,17.0,1.0,1.0,7.0,1.0,1.0,16.0,1.0,1.0,29.0,1.0,1.0,15.0,1.0,1.0,14.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,22.0,1.0,1.0,13.0,1.0,1.0,21.0,1.0,1.0,9.0,1.0,1.0,18.0,1.0,1.0,14.0,1.0,1.0,13.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,24.0,1.0,1.0,9.0,1.0,1.0,17.0,1,1,15.0,1.0,1.0,21.0
32,1.0,1.0,16.0,1.0,1.0,14.0,1.0,1.0,24.0,1.0,1.0,14.0,1.0,1.0,20.0,1.0,1.0,18.0,1.0,1.0,11.0,1.0,1.0,14.0,1.0,1.0,17.0,1.0,1.0,7.0,1.0,1.0,16.0,1.0,1.0,29.0,1.0,1.0,15.0,1.0,1.0,14.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,22.0,1.0,1.0,13.0,1.0,1.0,21.0,1.0,1.0,9.0,1.0,1.0,18.0,1.0,1.0,14.0,1.0,1.0,13.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,24.0,1.0,1.0,9.0,1.0,1.0,17.0,1.0,1.0,15.0,1,1,17.0,1.0,1.0,29.0
33,1.0,1.0,14.0,1.0,1.0,24.0,1.0,1.0,14.0,1.0,1.0,20.0,1.0,1.0,18.0,1.0,1.0,11.0,1.0,1.0,14.0,1.0,1.0,17.0,1.0,1.0,7.0,1.0,1.0,16.0,1.0,1.0,29.0,1.0,1.0,15.0,1.0,1.0,14.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,22.0,1.0,1.0,13.0,1.0,1.0,21.0,1.0,1.0,9.0,1.0,1.0,18.0,1.0,1.0,14.0,1.0,1.0,13.0,1.0,1.0,10.0,1.0,1.0,16.0,1.0,1.0,24.0,1.0,1.0,9.0,1.0,1.0,17.0,1.0,1.0,15.0,1.0,1.0,17.0,1,1,24.0,1.0,1.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182405,50.0,10.0,79.0,50.0,10.0,89.0,50.0,10.0,96.0,50.0,10.0,92.0,50.0,10.0,102.0,50.0,10.0,60.0,50.0,10.0,81.0,50.0,10.0,79.0,50.0,10.0,78.0,50.0,10.0,97.0,50.0,10.0,93.0,50.0,10.0,96.0,50.0,10.0,74.0,50.0,10.0,70.0,50.0,10.0,84.0,50.0,10.0,90.0,50.0,10.0,94.0,50.0,10.0,97.0,50.0,10.0,107.0,50.0,10.0,61.0,50.0,10.0,84.0,50.0,10.0,72.0,50.0,10.0,89.0,50.0,10.0,97.0,50.0,10.0,97.0,50.0,10.0,99.0,50.0,10.0,63.0,50.0,10.0,79.0,50.0,10.0,80.0,50,10,82.0,50.0,10.0,63.0
182406,50.0,10.0,89.0,50.0,10.0,96.0,50.0,10.0,92.0,50.0,10.0,102.0,50.0,10.0,60.0,50.0,10.0,81.0,50.0,10.0,7

In [97]:
columns_to_drop = [('%s(t+%d)' % (col, lag)) for col in ['item', 'store']]
for i in range(window, 0, -1):
    columns_to_drop += [('%s(t-%d)' % (col, i)) for col in ['item', 'store']]
series.drop(columns_to_drop, axis=1, inplace=True)
series.drop(['item(t)', 'store(t)'], axis=1, inplace=True)

In [98]:
series

,sales(t-29),sales(t-28),sales(t-27),sales(t-26),sales(t-25),sales(t-24),sales(t-23),sales(t-22),sales(t-21),sales(t-20),sales(t-19),sales(t-18),sales(t-17),sales(t-16),sales(t-15),sales(t-14),sales(t-13),sales(t-12),sales(t-11),sales(t-10),sales(t-9),sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t),sales(t+90)
29,19.0,15.0,10.0,16.0,14.0,24.0,14.0,20.0,18.0,11.0,14.0,17.0,7.0,16.0,29.0,15.0,14.0,10.0,16.0,22.0,13.0,21.0,9.0,18.0,14.0,13.0,10.0,16.0,24.0,9.0,33.0
30,15.0,10.0,16.0,14.0,24.0,14.0,20.0,18.0,11.0,14.0,17.0,7.0,16.0,29.0,15.0,14.0,10.0,16.0,22.0,13.0,21.0,9.0,18.0,14.0,13.0,10.0,16.0,24.0,9.0,17.0,15.0
31,10.0,16.0,14.0,24.0,14.0,20.0,18.0,11.0,14.0,17.0,7.0,16.0,29.0,15.0,14.0,10.0,16.0,22.0,13.0,21.0,9.0,18.0,14.0,13.0,10.0,16.0,24.0,9.0,17.0,15.0,21.0
32,16.0,14.0,24.0,14.0,20.0,18.0,11.0,14.0,17.0,7.0,16.0,29.0,15.0,14.0,10.0,16.0,22.0,13.0,21.0,9.0,18.0,14.0,13.0,10.0,16.0,24.0,9.0,17.0,15.0,17.0,29.0
33,14.0,24.0,14.0,20.0,18.0,11.0,14.0,17.0,7.0,16.0,29.0,15.0,14.0,10.0,16.0,22.0,13.0,21.0,9.0,18.0,14.0,13.0,10.0,16.0,24.0,9.0,17.0,15.0,17.0,24.0,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
182405,79.0,89.0,96.0,92.0,102.0,60.0,81.0,79.0,78.0,97.0,93.0,96.0,74.0,70.0,84.0,90.0,94.0,97.0,107.0,61.0,84.0,72.0,89.0,97.0,97.0,99.0,63.0,79.0,80.0,82.0,63.0
182406,89.0,96.0,92.0,102.0,60.0,81.0,79.0,78.0,97.0,93.0,96.0,74.0,70.0,84.0,90.0,94.0,97.0,107.0,61.0,84.0,72.0,89.0,97.0,97.0,99.0,63.0,79.0,80.0,82.0,90.0,59.0
182407,96.0,92.0,102.0,60.0,81.0,79.0,78.0,97.0,93.0,96.0,74.0,70.0,84.0,90.0,94.0,97.0,107.0,61.0,84.0,72.0,89.0,97.0,97.0,99.0,63.0,79.0,80.0,82.0,90.0,103.0,74.0
182408,92.0,102.0,60.0,81.0,79.0,78.0,97.0,93.0,96.0,74.0,70.0,84.0,90.0,94.0,97.0,107.0,61.0,84.0,72.0,89.0,97.0,97.0,99.0,63.0,79.0,80.0,82.0,90.0,103.0,99.0,62.0


In [99]:
labels_col = 'sales(t+%d)' % lag_size
labels = series[labels_col]
series = series.drop(labels_col, axis=1)

In [100]:
X_train, X_valid, Y_train, Y_valid = train_test_split(series, labels.values, test_size=0.4, random_state=0)
print('Train set shape', X_train.shape)
print('Validation set shape', X_valid.shape)
X_train.head()

Train set shape (100746, 30)
Validation set shape (67164, 30)


,sales(t-29),sales(t-28),sales(t-27),sales(t-26),sales(t-25),sales(t-24),sales(t-23),sales(t-22),sales(t-21),sales(t-20),sales(t-19),sales(t-18),sales(t-17),sales(t-16),sales(t-15),sales(t-14),sales(t-13),sales(t-12),sales(t-11),sales(t-10),sales(t-9),sales(t-8),sales(t-7),sales(t-6),sales(t-5),sales(t-4),sales(t-3),sales(t-2),sales(t-1),sales(t)
18801,97.0,111.0,90.0,115.0,123.0,70.0,99.0,74.0,107.0,108.0,121.0,136.0,82.0,106.0,81.0,80.0,108.0,131.0,115.0,86.0,85.0,95.0,123.0,109.0,127.0,132.0,87.0,101.0,102.0,114.0
160385,38.0,43.0,43.0,55.0,47.0,51.0,38.0,41.0,37.0,59.0,37.0,36.0,48.0,35.0,42.0,40.0,43.0,54.0,56.0,64.0,41.0,38.0,38.0,53.0,53.0,45.0,44.0,24.0,30.0,37.0
73123,55.0,45.0,41.0,46.0,47.0,36.0,30.0,46.0,41.0,42.0,55.0,64.0,30.0,38.0,36.0,38.0,38.0,40.0,46.0,43.0,38.0,36.0,40.0,50.0,44.0,44.0,40.0,38.0,50.0,49.0
90428,139.0,157.0,85.0,99.0,136.0,110.0,121.0,123.0,147.0,91.0,112.0,110.0,104.0,115.0,143.0,156.0,95.0,88.0,106.0,128.0,130.0,128.0,128.0,95.0,116.0,110.0,117.0,118.0,129.0,132.0
167151,86.0,58.0,88.0,87.0,114.0,113.0,64.0,76.0,87.0,81.0,78.0,95.0,100.0,63.0,77.0,80.0,103.0,64.0,71.0,74.0,55.0,66.0,59.0,53.0,63.0,59.0,77.0,39.0,56.0,62.0


In [101]:
epochs=40
batch=256
lr=0.0003
adam = optimizers.Adam(lr)

In [102]:
model_mlp = Sequential()
model_mlp.add(Dense(100,activation='relu',input_dim=X_train.shape[1]))
model_mlp.add(Dense(1))
model_mlp.compile(loss='mse', optimizer=adam)
model_mlp.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               3100      
                                                                 
 dense_1 (Dense)             (None, 1)                 101       
                                                                 
Total params: 3201 (12.50 KB)
Trainable params: 3201 (12.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [103]:
mlp_history = model_mlp.fit(X_train.values, Y_train, validation_data=(X_valid.values, Y_valid), epochs=epochs)

Epoch 1/40
3149/3149 [==============================] - 15s 3ms/step - loss: 422.9135 - val_loss: 385.3469
Epoch 2/40
3149/3149 [==============================] - 10s 3ms/step - loss: 372.5714 - val_loss: 371.3925
Epoch 3/40
3149/3149 [==============================] - 10s 3ms/step - loss: 360.9786 - val_loss: 376.8116
Epoch 4/40
3149/3149 [==============================] - 10s 3ms/step - loss: 357.6475 - val_loss: 386.4633
Epoch 5/40
3149/3149 [==============================] - 10s 3ms/step - loss: 355.8115 - val_loss: 355.0626
Epoch 6/40
3149/3149 [==============================] - 10s 3ms/step - loss: 354.3533 - val_loss: 354.7006
Epoch 7/40
3149/3149 [==============================] - 10s 3ms/step - loss: 352.8069 - val_loss: 357.6657
Epoch 8/40
3149/3149 [==============================] - 10s 3ms/step - loss: 351.2851 - val_loss: 350.0694
Epoch 9/40
3149/3149 [==============================] - 10s 3ms/step - loss: 351.0515 - val_loss: 350.3522
Epoch 10/40
3149/3149 [==============

In [104]:
X_train_series = X_train.values.reshape((X_train.shape[0], X_train.shape[1], 1))
X_valid_series = X_valid.values.reshape((X_valid.shape[0], X_valid.shape[1], 1))
print('Train set shape', X_train_series.shape)
print('Validation set shape', X_valid_series.shape)

Train set shape (100746, 30, 1)
Validation set shape (67164, 30, 1)


In [107]:
model_cnn = Sequential()
model_cnn.add(Conv1D(filters=64,kernel_size=2,activation='relu',input_shape=(X_train_series.shape[1], X_train_series.shape[2])))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(50, activation='relu'))
model_cnn.add(Dense(1))
model_cnn.compile(loss='mse', optimizer = tf.keras.optimizers.legacy.Adam())
model_cnn.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_1 (Conv1D)           (None, 29, 64)            192       
                                                                 
 max_pooling1d_1 (MaxPoolin  (None, 14, 64)            0         
 g1D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 896)               0         
                                                                 
 dense_4 (Dense)             (None, 50)                44850     
                                                                 
 dense_5 (Dense)             (None, 1)                 51        
                                                                 
Total params: 45093 (176.14 KB)
Trainable params: 45093 (176.14 KB)
Non-trainable params: 0 (0.00 Byte)
________________

In [108]:
cnn_history = model_cnn.fit(X_train_series, Y_train, validation_data=(X_valid_series, Y_valid), epochs=epochs, verbose=2)

Epoch 1/40
3149/3149 - 14s - loss: 415.6742 - val_loss: 415.2034 - 14s/epoch - 5ms/step
Epoch 2/40
3149/3149 - 10s - loss: 385.5787 - val_loss: 380.6190 - 10s/epoch - 3ms/step
Epoch 3/40
3149/3149 - 10s - loss: 370.9930 - val_loss: 375.2086 - 10s/epoch - 3ms/step
Epoch 4/40
3149/3149 - 10s - loss: 365.3181 - val_loss: 414.8150 - 10s/epoch - 3ms/step
Epoch 5/40
3149/3149 - 10s - loss: 362.2473 - val_loss: 356.5313 - 10s/epoch - 3ms/step
Epoch 6/40
3149/3149 - 10s - loss: 360.5349 - val_loss: 354.1996 - 10s/epoch - 3ms/step
Epoch 7/40
3149/3149 - 10s - loss: 358.5671 - val_loss: 359.9611 - 10s/epoch - 3ms/step
Epoch 8/40
3149/3149 - 10s - loss: 357.8265 - val_loss: 351.8721 - 10s/epoch - 3ms/step
Epoch 9/40
3149/3149 - 9s - loss: 357.9178 - val_loss: 351.8765 - 9s/epoch - 3ms/step
Epoch 10/40
3149/3149 - 10s - loss: 356.2557 - val_loss: 352.2311 - 10s/epoch - 3ms/step
Epoch 11/40
3149/3149 - 10s - loss: 355.4091 - val_loss: 351.0654 - 10s/epoch - 3ms/step
Epoch 12/40
3149/3149 - 10s - lo

In [110]:
mlp_train_pred = model_mlp.predict(X_train.values)
mlp_valid_pred = model_mlp.predict(X_valid.values)
print('Train rmse:', np.sqrt(mean_squared_error(Y_train, mlp_train_pred)))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_valid, mlp_valid_pred)))

2099/2099 [==============================] - 3s 1ms/step
Train rmse: 18.331080647111317
Validation rmse: 18.459227256808926


In [109]:
cnn_train_pred = model_cnn.predict(X_train_series)
cnn_valid_pred = model_cnn.predict(X_valid_series)
print('Train rmse:', np.sqrt(mean_squared_error(Y_train, cnn_train_pred)))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_valid, cnn_valid_pred)))

2099/2099 [==============================] - 3s 2ms/step
Train rmse: 18.431497253520135
Validation rmse: 18.561157134444798
